In [1]:
import httpx
import asyncio
import json

LOGIN_URL = 'https://awm.infeo.at/api/getsecuretoken'
TRACKINGS_URL = 'https://awm.infeo.at/api/trackings/'
TRACKINGS_MANY_URL ='https://awm.infeo.at/api/trackings/many'

LOCAL_FILE = "trackings.json"
DETAILED_FILE = "pointsdata.json"

USERNAME = 'flr#hefel@infeo'
PASSWORD = 'Infeo_2023'
GRANT_TYPE = "password" 

async def get_access_token():
    async with httpx.AsyncClient() as client:
        
        headers = {"Content-Type": "application/x-www-form-urlencoded"}
        data = {
            "username": USERNAME,
            "password": PASSWORD,
            "grant_type": GRANT_TYPE
        }
        response = await client.post(LOGIN_URL, data=data, headers=headers)
        
        print(response)
        
        if response.status_code == 200:
            return response.json().get("access_token")  
        else:
            print(f"Failed to authenticate: {response.status_code}, {response.text}")
            return None

async def get_trackings(access_token):
    headers = {"Authorization": f"Bearer {access_token}"}
    
    async with httpx.AsyncClient() as client:
        response = await client.get(TRACKINGS_URL, headers=headers)
        
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Failed to get trackings: {response.status_code}, {response.text}")
            return None
        
async def get_all_trackings(access_token):
    headers = {"Authorization": f"Bearer {access_token}"}
    start_index = 0
    count = 10000  
    all_trackings = []

    async with httpx.AsyncClient() as client:
        while True:
            url = f"{TRACKINGS_URL}?startIndex={start_index}&count={count}&orderBy=-recorded&isExported=false&isExportedFlag=0&noExportFlag=0&isExportable=false&isValid=false&isInvalid=false&validityUnknown=false"
            response = await client.get(url, headers=headers)
            
            if response.status_code == 200:
                data = response.json()
                items = data.get("items", []) 
                all_trackings.extend(items)
                
                print(len(all_trackings))
                
                if len(items) < count:
                    break  
                
                start_index += count
            else:
                print(f"Failed to get trackings: {response.status_code}, {response.text}")
                break
            
    with open(LOCAL_FILE, "w", encoding="utf-8") as f:
        json.dump(all_trackings, f, indent=4)
    
    print(f"Data saved to {LOCAL_FILE}")
    return all_trackings

async def get_detailed_trackings(access_token):
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }
    batch_size = 15
    
    with open(LOCAL_FILE, "r", encoding="utf-8") as f:
        trackings = json.load(f)
    
    tracking_ids = [t["id"] for t in trackings] 
    
    try:
        with open(DETAILED_FILE, "r", encoding="utf-8") as f:
            saved_data = json.load(f)
            processed_ids = {item["id"] for item in saved_data}
    except (FileNotFoundError, json.JSONDecodeError):
        saved_data = []
        processed_ids = set()
    
    async with httpx.AsyncClient() as client:
        for i in range(0, len(tracking_ids), batch_size):
            batch = [tid for tid in tracking_ids[i:i + batch_size] if tid not in processed_ids] 
            if not batch:
                continue 

            response = await client.put(TRACKINGS_MANY_URL, json=batch, headers=headers)
            
            if response.status_code == 200:
                batch_data = response.json()
                saved_data.extend(batch_data)
                
                with open(DETAILED_FILE, "w", encoding="utf-8") as f:
                    json.dump(saved_data, f, indent=4)
                
                print(f"Fetched {len(saved_data)} trackings so far...")
            else:
                print(f"Failed to get detailed trackings: {response.status_code}, {response.text}")
                break
    
    print(f"Final detailed data saved to {DETAILED_FILE}")
    return saved_data

    
async def main():
    token = await get_access_token()
    if token:
        # trackings = await get_all_trackings(token)
        await get_detailed_trackings(token)
        

await main()


<Response [200 OK]>
Fetched 15 trackings so far...
Fetched 30 trackings so far...
Fetched 45 trackings so far...
Fetched 60 trackings so far...
Fetched 75 trackings so far...
Fetched 90 trackings so far...


CancelledError: 